### IMPORTING STATEMENTS

In [1]:
import json
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import euclidean, jaccard
from scipy.stats import pearsonr
from nltk.stem import WordNetLemmatizer, SnowballStemmer
import nltk
from nltk.corpus import stopwords
import numpy as np
from collections import Counter
import re
from itertools import tee, islice
lemmatizer = WordNetLemmatizer()
stemmer = SnowballStemmer('english')
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import manhattan_distances
from sklearn.metrics import jaccard_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_json("C:\\Users\\MGC\\Downloads\\assignment\\News_Category_Dataset_v3.json\\News_Category_Dataset_v3.json", lines=True)

In [3]:
df

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22
...,...,...,...,...,...,...
209522,https://www.huffingtonpost.com/entry/rim-ceo-t...,RIM CEO Thorsten Heins' 'Significant' Plans Fo...,TECH,Verizon Wireless and AT&T are already promotin...,"Reuters, Reuters",2012-01-28
209523,https://www.huffingtonpost.com/entry/maria-sha...,Maria Sharapova Stunned By Victoria Azarenka I...,SPORTS,"Afterward, Azarenka, more effusive with the pr...",,2012-01-28
209524,https://www.huffingtonpost.com/entry/super-bow...,"Giants Over Patriots, Jets Over Colts Among M...",SPORTS,"Leading up to Super Bowl XLVI, the most talked...",,2012-01-28
209525,https://www.huffingtonpost.com/entry/aldon-smi...,Aldon Smith Arrested: 49ers Linebacker Busted ...,SPORTS,CORRECTION: An earlier version of this story i...,,2012-01-28


In [4]:
df.category.unique()

array(['U.S. NEWS', 'COMEDY', 'PARENTING', 'WORLD NEWS', 'CULTURE & ARTS',
       'TECH', 'SPORTS', 'ENTERTAINMENT', 'POLITICS', 'WEIRD NEWS',
       'ENVIRONMENT', 'EDUCATION', 'CRIME', 'SCIENCE', 'WELLNESS',
       'BUSINESS', 'STYLE & BEAUTY', 'FOOD & DRINK', 'MEDIA',
       'QUEER VOICES', 'HOME & LIVING', 'WOMEN', 'BLACK VOICES', 'TRAVEL',
       'MONEY', 'RELIGION', 'LATINO VOICES', 'IMPACT', 'WEDDINGS',
       'COLLEGE', 'PARENTS', 'ARTS & CULTURE', 'STYLE', 'GREEN', 'TASTE',
       'HEALTHY LIVING', 'THE WORLDPOST', 'GOOD NEWS', 'WORLDPOST',
       'FIFTY', 'ARTS', 'DIVORCE'], dtype=object)

In [5]:
df.category[df.category=='THE WORLDPOST'] = 'WORLDPOST'
df.category[df.category=='GREEN'] = 'ENVIRONMENT'
df.category[df.category=='CULTURE & ARTS'] = 'ARTS'
df.category[df.category=='COMEDY'] = 'ENTERTAINMENT'
df.category[(df.category=='BLACK VOICES') | (df.category=='LATINO VOICES') | (df.category=='QUEER VOICES')] = 'VOICES'
df.category[df.category=='STYLE'] = 'STYLE & BEAUTY'
df.category[df.category=='ARTS & CULTURE'] = 'ARTS'
df.category[df.category=='COLLEGE'] = 'EDUCATION'
df.category[df.category=='SCIENCE'] = 'TECH'
df.category[df.category=='WEDDINGS'] = 'GOOD NEWS'
df.category[df.category=='TASTE'] = 'FOOD & DRINK'
df.category[(df.category=='PARENTING') | (df.category=='FIFTY')] = 'PARENTS'
df.category[df.category=='WORLD NEWS'] = 'WORLDPOST'

In [6]:
def textClean(text):
    text = re.sub("[^a-zA-Z]", ' ',text).strip()
    text = nltk.word_tokenize(text.lower())
    stops = set(stopwords.words("english"))
    stops = list(stops) + ['amp']
    text = " ".join([lemmatizer.lemmatize(w) for w in text if w not in stops])
    return(text)

### CLEANING HEADLINE AND DESCRIPTION

In [7]:
%%time
df['Clean_headline'] = df.headline.map(textClean)
df['Clean_description'] = df.short_description.map(textClean)

CPU times: total: 2min 15s
Wall time: 5min 13s


In [8]:
df[df.Clean_headline.str.contains('box office')].category.value_counts()

ENTERTAINMENT    260
VOICES             6
PARENTS            1
Name: category, dtype: int64

In [9]:
df.drop(["short_description", "headline"], axis=1, inplace=True)

### FITTING TEXT DATA TO TFIDF VECTORIZER

In [10]:
# Create a TF-IDF vectorizer
vectorizer = TfidfVectorizer()

df['text_data'] = df['Clean_headline'] + ' ' + df['Clean_description']
# Fit and transform the textual data
textual_data = df['text_data'][0:10000]  # Assuming 'Clean_description' contains the textual data
tfidf_matrix = vectorizer.fit_transform(textual_data)

# Compute cosine similarity
cosine_similarities = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Compute Jaccard similarity
binary_matrix = (tfidf_matrix > 0).astype(int)
jaccard_similarities = jaccard_score(binary_matrix.toarray(), binary_matrix.toarray(), average=None)

# Find the most similar data using each similarity algorithm
most_similar_cosine = df.iloc[cosine_similarities.argmax(axis=1)]
most_similar_jaccard = df.iloc[jaccard_similarities.argmax(axis=0)]



###  Finding the most similar data points using cosine similarity
 

In [11]:
query_index = 0  # Index of the query data point
most_similar_indices = np.argsort(-cosine_similarities[query_index])  # Sort in descending order
most_similar_data = df["text_data"][most_similar_indices]

print("Most Similar Data Points (Cosine Similarity):")
print(most_similar_data)


Most Similar Data Points (Cosine Similarity):
0       million american roll sleeve omicron targeted ...
2384    fda cdc say fully vaccinated need booster shot...
2156    common covid booster shot question answered la...
1673    virus surge colorado open covid booster adult ...
1497    fauci say want optimally protected get covid b...
                              ...                        
4172    chadwick boseman helped create iconic black pa...
4171    faith leader endorse joe biden kamala harris w...
4169    fonio try nutrient packed ancient grain quinoa...
4182    massachusetts primary could affect puerto rica...
9999    facebook let user see site tracking able stop ...
Name: text_data, Length: 10000, dtype: object


### Finding the most similar data points using Jaccard similarity

In [16]:
query_index = 2  # Index of the query data point
most_similar_indices = np.argsort(-jaccard_similarities[query_index])[0:]  # Sort in descending order
most_similar_data = df["text_data"][most_similar_indices]

print("Most Similar Data Points (Jaccard Similarity):")
print(most_similar_data)


Most Similar Data Points (Jaccard Similarity):
0    million american roll sleeve omicron targeted ...
Name: text_data, dtype: object


### Finding the most similar data points using Euclidean similarity


In [13]:
from sklearn.metrics.pairwise import pairwise_distances

# Compute Euclidean distances
euclidean_distances = pairwise_distances(tfidf_matrix.toarray(), metric='euclidean')

# Find the most similar data points using Euclidean distance
query_index = 0  # Index of the query data point
most_similar_indices_euclidean = np.argsort(euclidean_distances[query_index])  # Sort in ascending order
most_similar_data_euclidean = df["text_data"].iloc[most_similar_indices_euclidean]

print("Most Similar Data Points (Euclidean Distance):")
print(most_similar_data_euclidean)


Most Similar Data Points (Euclidean Distance):
0       million american roll sleeve omicron targeted ...
2384    fda cdc say fully vaccinated need booster shot...
2156    common covid booster shot question answered la...
1673    virus surge colorado open covid booster adult ...
1497    fauci say want optimally protected get covid b...
                              ...                        
6648    dairy farm investigation undercover footage re...
6647    california santa anita racetrack refuse close ...
2822    zendaya oscar dress stunning made bigger fashi...
6643    men sentenced life prison rape murder year old...
9999    facebook let user see site tracking able stop ...
Name: text_data, Length: 10000, dtype: object


### Finding the most similar data points using Manhattan similarity


In [17]:
from sklearn.metrics.pairwise import manhattan_distances

# Calculate the Manhattan distances between the query data and all other data points
manhattan_distances = manhattan_distances(tfidf_matrix, tfidf_matrix)
query_index=0
# Sort the distances in ascending order and get the indices of the most similar data points
most_similar_indices_manhattan = np.argsort(manhattan_distances[query_index])

# Get the corresponding data points from the dataset
most_similar_data_manhattan = df["text_data"].iloc[most_similar_indices_manhattan]

# Print the most similar data points (Manhattan Distance)
print("Most Similar Data Points (Manhattan Distance):")
print(most_similar_data_manhattan)


Most Similar Data Points (Manhattan Distance):
0       million american roll sleeve omicron targeted ...
9042    trump knew scary amount appearance bill gate d...
2156    common covid booster shot question answered la...
478                            herschel walker say state 
2432                       dress want twirl go twirl pick
                              ...                        
230     sinema took wall street money killing tax inve...
4923    adam sandler pop pete davidson snl rap going q...
7475    trailer little movie drop twitter already cele...
6162    deal brexit could lead shortage medicine food ...
3139    dozen leading hong kong pro democracy campaign...
Name: text_data, Length: 10000, dtype: object
